<a href="https://colab.research.google.com/github/chrismartel/ECSE415-Project-G7/blob/main/tracking/tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Optical Flow Method (Lucas-Kanade)

###Tracking Methods from OpenCV

In [ ]:
import cv2

# tracking methods available in OpenCV:

# tracker = cv2.TrackerBoosting_create()
# tracker = cv2.TrackerMIL_create()
# tracker = cv2.TrackerKCF_create()
# tracker = cv2.TrackerTLD_create()
# tracker = cv2.TrackerMedianFlow_create()
# tracker = cv2.TrackerGOTURN_create()
# tracker = cv2.TrackerMOSSE_create()
# tracker = cv2.TrackerCSRT_create()